#setup
- connect to google drive in this step

In [1]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [4]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [5]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [6]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#choose video

In [153]:
video_name = 'PZ71_1'

female_side_mat = scipy.io.loadmat(project_path + "/temp/" + video_name + "_female_side.mat")
female_side_vec = female_side_mat['croprect'][0]
male_side_mat = scipy.io.loadmat(project_path + "/temp/" + video_name + "_male_side.mat")
male_side_vec = male_side_mat['croprect'][0]

h5_file = pd.read_hdf(project_path +'/temp/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
[nframes, ncols] = h5_file.shape

#analysis setup

In [154]:
# get all data ready for accessing as needed later on
mouse1_feature_points = {}
mouse1_feature_points['snout'] = get_data('mus1', 'snout', h5_file)
mouse1_feature_points['shoulder'] = get_data('mus1', 'shoulder', h5_file)
mouse1_feature_points['spine1'] = get_data('mus1', 'spine1', h5_file)
mouse1_feature_points['spine2'] = get_data('mus1', 'spine2', h5_file)
mouse1_feature_points['spine3'] = get_data('mus1', 'spine3', h5_file)
mouse1_feature_points['spine4'] = get_data('mus1', 'spine4', h5_file)
mouse1_feature_points['tailbase'] = get_data('mus1', 'tailbase', h5_file)

mouse2_feature_points = {}
mouse2_feature_points['snout'] = get_data('mus2', 'snout', h5_file)
mouse2_feature_points['shoulder'] = get_data('mus2', 'shoulder', h5_file)
mouse2_feature_points['spine1'] = get_data('mus2', 'spine1', h5_file)
mouse2_feature_points['spine2'] = get_data('mus2', 'spine2', h5_file)
mouse2_feature_points['spine3'] = get_data('mus2', 'spine3', h5_file)
mouse2_feature_points['spine4'] = get_data('mus2', 'spine4', h5_file)
mouse2_feature_points['tailbase'] = get_data('mus2', 'tailbase', h5_file)

mouse3_feature_points = {}
mouse3_feature_points['snout'] = get_data('mus3', 'snout', h5_file)
mouse3_feature_points['shoulder'] = get_data('mus3', 'shoulder', h5_file)
mouse3_feature_points['spine1'] = get_data('mus3', 'spine1', h5_file)
mouse3_feature_points['spine2'] = get_data('mus3', 'spine2', h5_file)
mouse3_feature_points['spine3'] = get_data('mus3', 'spine3', h5_file)
mouse3_feature_points['spine4'] = get_data('mus3', 'spine4', h5_file)
mouse3_feature_points['tailbase'] = get_data('mus3', 'tailbase', h5_file)


In [155]:
# male_side_vec = [x, y, width, height]; female_side_vec = [x, y, width, height]
relevant_area = female_side_vec.copy()
# relevant area is female_side_vec + male_side_vec size exactly right next to each other, w 50 buffer
relevant_area[2] = female_side_vec[2] + male_side_vec[2] + 50

In [156]:
def torso_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine1'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine2'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine3'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['spine4'].loc[[i]]):
    return 1
  elif within_area(area_vector, feature_points['tailbase'].loc[[i]]):
    return 1
  else:
    return 0

In [157]:
def snout_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['snout'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

In [158]:
def shoulder_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['shoulder'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

In [159]:
def tailbase_in_area(area_vector, mouse_num, i):
  # finds if any part of torso at all is in area
  if mouse_num == 1:
    feature_points = mouse1_feature_points
  elif mouse_num == 2:
    feature_points = mouse2_feature_points
  elif mouse_num == 3:
    feature_points = mouse3_feature_points

  if within_area(area_vector, feature_points['tailbase'].loc[[i]]):
    result = 1
  else:
    result = 0
  return result

#putative intros

Note: dlc origin is in the top left; x goes to the right, y goes down

In [160]:
# 5 min
behavior = pd.DataFrame(index=range(36000),columns=['behavior'])

for i in range(0, 36000):
  if torso_in_area(female_side_vec, 1, i) or torso_in_area(female_side_vec, 2, i) or torso_in_area(female_side_vec, 3, i):
    event_name = 0

  elif (tailbase_in_area(male_side_vec, 1, i) and tailbase_in_area(male_side_vec, 2, i)) or \
    (tailbase_in_area(male_side_vec, 1, i) and tailbase_in_area(male_side_vec, 3, i)) or \
    (tailbase_in_area(male_side_vec, 2, i) and tailbase_in_area(male_side_vec, 3, i)):
    event_name = 0
    if mouse1_feature_points['tailbase'].loc[[i]].equals(mouse2_feature_points["tailbase"].loc[[i]]) or \
    mouse1_feature_points['tailbase'].loc[[i]].equals(mouse3_feature_points["tailbase"].loc[[i]]) or \
    mouse2_feature_points['tailbase'].loc[[i]].equals(mouse3_feature_points["tailbase"].loc[[i]]):
      event_name = 1

  else:
    event_name = 1

  behavior["behavior"][i] = event_name

In [161]:
import math
smoothed_behavior = np.zeros((2, 36000))
for i in range(30, 35670):
  avg_behavior = behavior["behavior"][i-30:i+30].mean()
  if avg_behavior > 0.9:
    smoothed_behavior[1, i] = 1


In [162]:
intro_events = pd.DataFrame(index=range(nframes - 1), columns=['events_index', 'minutes', 'seconds', 'behavior_events'])

events_n = 0
for i in range(30, 35970):
  if smoothed_behavior[1, i] == 1 and smoothed_behavior[1, i-1] == 0:
    intro_events["events_index"][events_n] = i
    intro_events["minutes"][events_n] = i / (30*60)
    intro_events["seconds"][events_n] = round(i / 30) % 60
    intro_events["behavior_events"][events_n] = 1
    events_n = events_n + 1
print(events_n)
intro_events = intro_events.dropna(how='all')

with open(output_dir + video_name + '_intro_events.pickle', 'wb') as f:
    pickle.dump(intro_events, f)

39


In [163]:
import csv
pickle_data = intro_events
with open(output_dir + video_name +'_intro_events.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(pickle_data.columns)
    for i in range(0, pickle_data.shape[0]):
      # write the data
      writer.writerow(pickle_data.values[i,:])

#make labelled behavior video

In [22]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 36000
for video in [project_path + "/videos/PZ71_1.avi"]:
#for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior = smoothed_behavior

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    if behavior[1, i] == 1:
      text_color = (0,255,0)
      cv2.putText(frame, str('intro'), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
      
    cv2.putText(frame, str(count), (25, frame_height), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
    out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(168, 288, 3)
